In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [6]:
KEY=os.getenv("OPEN_API_KEY")